In [0]:
import tensorflow as tf
import cv2 as cv
import os
import numpy as np
import glob
from random import shuffle
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dataset = pd.read_csv('/content/drive/My Drive/mrdata.csv')


In [0]:
sent=[]
for l in dataset['Phrase'].values:
  sent.append(str(l))
print(sent)  


['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .', 'A series of escapades demonstrating the adage that what is good for the goose', 'A series', 'A', 'series', 'of escapades demonstrating the adage that what is good for the goose', 'of', 'escapades demonstrating the adage that what is good for the goose', 'escapades', 'demonstrating the adage that what is good for the goose', 'demonstrating the adage', 'demonstrating', 'the adage', 'the', 'adage', 'that what is good for the goose', 'that', 'what is good for the goose', 'what', 'is good for the goose', 'is', 'good for the goose', 'good', 'for the goose', 'for', 'the goose', 'goose', 'is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .', 'is also good for the gander , some of which occasionally amuses but none of which amounts to much of a s

In [0]:
token = tf.keras.preprocessing.text.Tokenizer(num_words=3000,lower=True)
token.fit_on_texts(sent)
sequence= token.texts_to_sequences(sent)

In [0]:
w2i=token.word_index
data=tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=100)

In [0]:
w2v={}
with open(os.path.join('/content/drive/My Drive/glove.6B.50d.txt')) as f :
  for l in f:
    v=l.split()
    w=v[0]
    vec=np.asarray(v[1:],dtype = 'float32')
    w2v[w] =vec
    

In [0]:
n_o_w = min(3000,len(w2i)+1)
embedding_matrix = np.zeros((n_o_w,50))

In [0]:
for key,value in w2i.items():
  if value<3000: 
        if w2v.get(key) is not None:
            embedding_matrix[value] = w2v.get(key)

In [0]:
embedding_layer = tf.keras.layers.Embedding(input_dim=n_o_w,output_dim=50,weights = [embedding_matrix],input_length=100,trainable = False)


In [0]:
# ohe = OneHotEncoder()
# t_ohe = ohe.fit_transform(np.array(dataset['Sentiment'].values).reshape(-1,1)).toarray()
# print(t_ohe.shape)
t_ohe=np.array(dataset['Sentiment'].values)
t_ohe=np.eye(5,dtype=int)[t_ohe]
data=data.tolist()

In [0]:
# ls,ls1,otp,otp1 = train_test_split(data,t_ohe)

otp1=t_ohe[52427:]
otp=t_ohe[:52427]
ls1=data[52427:]
ls=data[:52427]
ls1=np.asarray(ls1)
otp1=np.asarray(otp1)
l_of_data=len(ls)

In [0]:
ip = tf.placeholder(shape =(None,100), dtype=tf.float32)
x = embedding_layer(ip)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True))(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
logits = tf.keras.layers.Dense(5)(x)
output = tf.keras.layers.Dense(5,activation="sigmoid")(x)
y_true= tf.placeholder(shape =(None,5), dtype=tf.float32)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y_true,logits=output))


optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)


train_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()


sess = tf.Session()

sess.run(init)

In [0]:
for ep in range(100):
    index = np.arange(0 ,l_of_data)
    np.random.shuffle(index)
    index = index[:64]
    b_x,b_y= np.asarray([ls[ i] for i in index]),np.asarray([otp[ i] for i in index]) 
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y_true, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    _,l,acc= sess.run([train_op,loss,accuracy],feed_dict={ip:b_x, y_true:b_y})
    if(ep%10==0):
        print("epoch :",ep)
        print('Loss: ',l)
        print('training accuracy: ',acc)
l,acc= sess.run([loss,accuracy],feed_dict={ip:ls1, y_true:otp1})
print('testing Loss: ',l)
print('testing accuracy: ',acc)

epoch : 0
Loss:  0.69315726
training accuracy:  0.453125
epoch : 10
Loss:  0.693143
training accuracy:  0.515625
epoch : 20
Loss:  0.69312245
training accuracy:  0.59375
epoch : 30
Loss:  0.6931639
training accuracy:  0.4375
epoch : 40
Loss:  0.69316417
training accuracy:  0.4375
epoch : 50
Loss:  0.6931186
training accuracy:  0.578125
epoch : 60
Loss:  0.6930979
training accuracy:  0.625
epoch : 70
Loss:  0.6931148
training accuracy:  0.5625
epoch : 80
Loss:  0.693178
training accuracy:  0.4375
epoch : 90
Loss:  0.6931489
training accuracy:  0.484375
testing Loss:  0.69313586
testing accuracy:  0.5104906
